In [ ]:

!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-aiplatform 1.54.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [ ]:

import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse


# set up gorp api key
os.environ["GROQ_API_KEY"] = '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'




def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))


In [ ]:
from langchain.tools import Tool

#upload the pdfs in data

In [ ]:
!mkdir data

# !gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

In [ ]:
from langchain_core.tools import tool
from langchain.tools import BaseTool

# Create OCR Tool

In [ ]:
class OCRTool():
    def __init__(self,img_path):
      self.img_path = img_path


    def run(self):
      # Prepare the payload
      # API endpoint and headers
      url = "https://pen-to-print-handwriting-ocr.p.rapidapi.com/recognize/"
      headers = {
    "X-RapidAPI-Key": "1ff515ad07mshab205ba81f47020p1c9e1bjsnde5813fcd36f",
    "X-RapidAPI-Host": "pen-to-print-handwriting-ocr.p.rapidapi.com"}

      # Read the image file as bytes
      with open(self.img_path, 'rb') as f:
          image_bytes = f.read()

      payload = {
          "Session": "string"
      }

      # Create the files dictionary with the image file
      files = {
          "srcImg": (self.img_path, image_bytes)
      }

      # Send the POST request
      response = requests.post(url, data=payload, files=files, headers=headers)

      # Check the response
      if response.status_code == 200:
          return (response.json().get('value'))
      else:
          print("OCR Error:", response.status_code)
          print(response.text)

#Model loading

In [ ]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

#example one

In [ ]:
from langchain.agents import initialize_agent


In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
ocr = OCRTool(img_path = '/content/product.png')
label = ocr.run()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nutrition specialist. You will be given label information from the back of a packaged product. You are supposed to describe how healthy or unhealthy the product might be based on the Indian standards of healthy ingridient levels. Answer in pointers. In each pointer, along with the consequences also mention the indian healthy standards. And give a final conclusion at the end."),
    ("user", f"{label}")
])

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
label

'zu re thore chips claim is in comparison\nwith previous pack of Doritos (Rs. 25/39g),\ncalculated on net quantity per rupee.\nSource Euromonitor International Limited;\npackaged Food; Nacho chip as per Tortilla\nhp dekgition, in retail value RSP terms, all\nretail channels, previous year data.\nDoritos is he Registered Trademark\nof PepsiCo, Inc.\nPROPRIETARY FOOD - NAMKEEN (15.1)\nINGREDIENTS: Corn (77%), Edible Vegetable Oi,\n(Palmolein), *Seasoning (Milk Solids, Refined\nWheat Flour (Maida), lodised Salt, Flavour\n(Natural and Nature Identical Flavouring\nSubstances), Cheese Powder, Tomato Powder,\n"Spices & Condiments, Anticaking Agent (551),\nColor (160c), Acidity Regulator (330)).\n*As flavouring agent. ~ Contains Onion and Garlic.\nALLERGEN ADVICE: Contains Milk, Wheat.\nNUTRITIONAL INFORMATION^\nServe Size - 20 g^^\nNutrients\nPer 100 g\nEnergy\n499 kcal\n5%\nProtein\n7.1 g\nCarbohydrate\n65.1 g\nTotal Sugars\n4.5 g\nAdded Sugars\n0.0 g\n0%\nTotal Fat\n23.4 g\n7%\nSaturated Fa

In [ ]:
print(chain.invoke({"input" : f"Ans"}))

Here's the analysis of the nutritional information of the Doritos Nacho Chips:

* **High Energy Content**: The product has 499 kcal per 100g, which is high in energy density. This can contribute to weight gain and obesity if consumed excessively. Indian healthy standards recommend limiting energy intake from snacks to 100-150 kcal per serving.

* **High in Carbohydrates**: The product contains 65.1g of carbohydrates per 100g, which is high. While some carbohydrates are necessary, excessive consumption can lead to insulin resistance and weight gain. Indian healthy standards recommend limiting carbohydrate intake from snacks to 20-30g per serving.

* **High in Total Fat**: The product contains 23.4g of total fat per 100g, which is high. High fat intake can contribute to weight gain, obesity, and increased risk of cardiovascular disease. Indian healthy standards recommend limiting total fat intake from snacks to 10-15g per serving.

* **High in Saturated Fat**: The product contains 9.0g o

#Vector store and DB

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../", glob="**/*.pdf", use_multithreading=True)
docs = loader.load()

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.9 MB/s eta 0:00:00


In [ ]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

#chat history aware

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input":  "her name is 3,  tell  me how langsmith is helpful"
})

#Langserve api

In [ ]:
!pip install "langserve[all]"